# Simulated Dyadic IBI Data

We generate three synthetic dyad recordings to demonstrate coupling, leader–follower dynamics,
and non-coupling in the windowed cross-correlation (WXC) tool.
All simulated signals are derived from a real IBI recording so they retain physiological realism.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import welch

plt.rcParams.update({
    'figure.dpi': 120,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
})

## Source data

We load person A's IBI series from a real recording. This is our **source signal** —
all simulated person-B series will be derived from it, ensuring the base physiology looks real.

In [ ]:
def load_ibi(path, clip_ms=(300, 990)):
    """Load IBI series from a Mindware xlsx export (sheet 'IBI Series').
    Row 0 is a segment marker and is skipped. Values outside clip_ms are removed.
    Upper bound kept below 1000 ms to stay within the WXC tool's valid range."""
    df = pd.read_excel(path, sheet_name='IBI Series', header=None)
    ibi = df.iloc[1:, 0].astype(float).values
    lo, hi = clip_ms
    mask = (ibi >= lo) & (ibi <= hi)
    n_removed = (~mask).sum()
    if n_removed:
        print(f'  {path}: removed {n_removed} outlier(s) outside [{lo}, {hi}] ms')
    return ibi[mask]

ibi_a = load_ibi('dyad_recording/HRV_a.xlsx')
ibi_b_real = load_ibi('dyad_recording/HRV_b.xlsx')   # kept for reference / condition 3

MEAN_B = ibi_b_real.mean()   # target mean HR for simulated person B (~644 ms, ~93 bpm)

for name, ibi in [('A (source)', ibi_a), ('B (real, reference)', ibi_b_real)]:
    dur = ibi.sum() / 1000
    print(f'Person {name}: n={len(ibi):4d}  mean={ibi.mean():.1f} ms '
          f'({60000/ibi.mean():.1f} bpm)  SD={ibi.std():.1f} ms  '
          f'duration={dur:.0f} s ({dur/60:.1f} min)')